In [5]:
import re
import sys
sys.path.append("..")
import lib
import numpy as np
from collections import deque

In [6]:
global MONKEYS
MONKEYS = []
global KGV
KGV = 0
global lambda_dict_example, lambda_dict_input
lambda_dict_example = [
    lambda x : 19*x,
    lambda x : x+6,
    lambda x : x*x,
    lambda x : x+3,
]

lambda_dict_input = [
    lambda x : 13*x,
    lambda x : x + 2,
    lambda x : x + 8,
    lambda x : x + 1,
    lambda x : 17*x,
    lambda x : x + 3,
    lambda x : x*x,
    lambda x : x + 6, 
]

class Monkey():
    def __init__(self):
        # use parse_monkey and parse_split_into_monkeys to create instances
        self.number = None
        self.worry_levels = [NotImplemented] # list of starting worry levels
        self.operation = NotImplemented # uses eval() for operation -> not recommended in general
        self.divisible_value = None # value division is tested by
        self.action_true = None # stores monkey the item is thrown to if test TRUE
        self.action_false = None # stores monkey the item is thrown to if test FALSE
        self.inspect_counter = 0 # stores how often the monkey inspects items
        self.relief_flag = None
        self.monkey_id = None
        return

    def __repr__(self):
        return f"\nMonkey {self.number}: {self.inspect_counter} / {self.worry_levels}"
    
    def append(self, worry_level : int):
        self.worry_levels.append(worry_level)
        return

    def inspect(self, idx : int, verbose : bool = False):
        #old = self.worry_levels[idx]
        print(f"  Monkey {self.number} inspects item with worry {self.worry_levels[idx]}") if verbose else None
        self.worry_levels[idx] = int(self.worry_levels[idx])
        self.worry_levels[idx] = self.operation(self.worry_levels[idx]) # monkey inspects
        print(f"    Worry level increases to {self.worry_levels[idx]}") if verbose else None
        global KGV # take lowest common multiple to facilitate calculation
        if not self.relief_flag:
            self.worry_levels[idx] = self.worry_levels[idx] % KGV
        self.inspect_counter += 1        
        return
    
    def throw(self, idx : int, verbose : bool = False):
        # monkey tests worry level and throws depending on that
        if self.worry_levels[idx] % self.divisible_value == 0:
            target_monkey_idx = self.action_true
            print(f"    Divisible by {self.divisible_value}, thrown to Monkey {target_monkey_idx}") if verbose else None
        else:
            target_monkey_idx = self.action_false
            print(f"    Not divisible by {self.divisible_value}, thrown to Monkey {target_monkey_idx}") if verbose else None
        global MONKEYS
        MONKEYS[target_monkey_idx].append(self.worry_levels[idx])
        # own worry levels list destroyed at end of take turn
        return
    
    def relief(self, idx : int, verbose : bool = False):
        self.worry_levels[idx] //= 3 # monkey gets bored
        print(f"    Monkey gets bored, worry level divided by 3 to {self.worry_levels[idx]}") if verbose else None
        return
    
    def take_turn(self, verbose : bool = False):
        for idx in range(len(self.worry_levels)):
            self.inspect(idx, verbose)
            self.relief(idx, verbose) if self.relief_flag else None
            self.throw(idx, verbose)
        self.worry_levels = [] # destroy worry level list
        return

def take_round(verbose : bool = False):
    # iterate through each monkey and make them take their turn
    global MONKEYS
    for i in range(len(MONKEYS)):
        print(f"\nExecute turn of monkey {i}...") if verbose else None
        MONKEYS[i].take_turn(verbose)
    return

def parse_monkey(input : str, is_example : bool, verbose : bool = False) -> Monkey:
    input_parts = re.split("\n", input)
    monkey = Monkey()
    monkey.monkey_id = int(input_parts[0][7])
    print(f"Parsed Monkey {monkey.monkey_id}:") if verbose else None

    # parse starting items
    regex = re.compile("\d+")
    result = regex.findall(input_parts[1])
    result = [int(res) for res in result]
    monkey.worry_levels = result
    print(f"Worry levels of starting items: {result}") if verbose else None

    if is_example:
        monkey.operation = lambda_dict_example[monkey.monkey_id]
    else:
        monkey.operation = lambda_dict_input[monkey.monkey_id]
    print(f"Operation: {monkey.operation}") if verbose else None

    # parse test
    regex = re.compile("Test: divisible by (\d+)$")
    result = regex.findall(input_parts[3])[0]
    result = int(result)
    monkey.divisible_value = result
    print(f"Test if divisible by {result}.") if verbose else None

    # parse true action
    regex = re.compile("If true: throw to monkey (\d+)$")
    result = regex.findall(input_parts[4])[0]
    result = int(result)
    monkey.action_true = result
    print(f"If true throw to {result}") if verbose else None

    # parse false action
    regex = re.compile("If false: throw to monkey (\d+)$")
    result = regex.findall(input_parts[5])[0]
    result = int(result)
    monkey.action_false = result
    print(f"If false throw to {result}\n") if verbose else None

    return monkey

def parse_split_into_monkeys(input : str, is_example : bool, relief_flag : bool = True, verbose : bool = False) -> None:
    regex = re.split("\n{2}", input)
    global MONKEYS
    MONKEYS = [parse_monkey(regex_part, is_example, verbose) for regex_part in regex]
    for i in range(len(MONKEYS)):
        MONKEYS[i].number = i
        MONKEYS[i].relief_flag = relief_flag
    return

In [14]:
def compute_partA(filename : str, n_rounds : int, is_example : bool, verbose : bool = False):
    input = lib.read_file_as_one(filename)
    parse_split_into_monkeys(input, is_example, True, False)

    for i in range(1, n_rounds+1):
        take_round(verbose)
        for monkey in MONKEYS:
            print(monkey) if verbose else None

    n_inspect = [monkey.inspect_counter for monkey in MONKEYS]
    n_inspect = sorted(n_inspect, reverse = True)
    print(f"Inspects: {n_inspect}")
    return n_inspect[0] * n_inspect[1]

def solve_partA():
    n_rounds = 20
    result = compute_partA("test_input.txt", n_rounds, True, False)
    print(f"Result of part A on test file : {result}")
    assert result == 10605, f"Part A faulty on test file... output = {result}"
    print("Part A works for test file, moving on to whole input...")
    result = compute_partA("input.txt", n_rounds, False, False)
    print(f"Answer: {result}")
    return

def compute_partB(filename : str, n_rounds : int, is_example : bool, verbose : bool = False):
    input = lib.read_file_as_one(filename)
    parse_split_into_monkeys(input, is_example, False, False)

    # compute KGV
    global KGV
    global MONKEYS
    list_of_divisibles = [monkey.divisible_value for monkey in MONKEYS]
    print(f"List of divisibles : {list_of_divisibles}")
    KGV = int(np.prod(list_of_divisibles))
    print(f"KGV = {KGV}")

    for monkey in MONKEYS:
        print(monkey) if verbose else None
    for i in range(1, n_rounds+1):
        take_round(verbose)
        if verbose:
            print(f"\n== Round {i} finished ==\n")
            for monkey in MONKEYS:
                print(monkey)
        if i % 1000 == 0:
            print(f"== After round {i} ==")
            for monkey in MONKEYS:
                print(monkey)
    n_inspect = [monkey.inspect_counter for monkey in MONKEYS]
    n_inspect = sorted(n_inspect, reverse = True)
    print(n_inspect)
    return n_inspect[0] * n_inspect[1]

def solve_partB():
    n_rounds = 10000
    result = compute_partB("test_input.txt", n_rounds, True, False)
    print(f"Result of part B on test file : {result}")
    assert result == 2713310158, f"Part B faulty on test file... output = {result}"
    print("Part B works for test file, moving on to whole input...")
    result = compute_partB("input.txt", n_rounds, False, False)
    print(f"Answer: {result}")
    return


In [15]:
solve_partA()

Inspects: [105, 101, 95, 7]
Result of part A on test file : 10605
Part A works for test file, moving on to whole input...
Inspects: [337, 333, 323, 318, 314, 302, 55, 25]
Answer: 112221


In [16]:
solve_partB()

List of divisibles : [23, 19, 13, 17]
KGV = 96577
== After round 1000 ==

Monkey 0: 5204 / [84464, 48934, 39396, 19275, 48307, 82374, 27591]

Monkey 1: 4792 / [12752, 69429, 31980]

Monkey 2: 199 / []

Monkey 3: 5192 / []
== After round 2000 ==

Monkey 0: 10419 / [70043, 40327, 18249, 24424, 25374]

Monkey 1: 9577 / [66693, 70664, 42468, 95193, 53621]

Monkey 2: 392 / []

Monkey 3: 10391 / []
== After round 3000 ==

Monkey 0: 15638 / [69378, 34665, 28737, 93584, 25203, 13056]

Monkey 1: 14358 / [48928, 42924, 49479, 58048]

Monkey 2: 587 / []

Monkey 3: 15593 / []
== After round 4000 ==

Monkey 0: 20858 / [16425, 18401, 77700, 17299, 33924]

Monkey 1: 19138 / [70037, 91393, 81950, 48301, 4544]

Monkey 2: 780 / []

Monkey 3: 20797 / []
== After round 5000 ==

Monkey 0: 26075 / [68637, 40688, 4335]

Monkey 1: 23921 / [18243, 20504, 60689, 34659, 6330, 47028, 84154]

Monkey 2: 974 / []

Monkey 3: 26000 / []
== After round 6000 ==

Monkey 0: 31294 / [83932, 65806, 29269, 25355]

Monkey 1: 